In [8]:
# cette cellule pour l'execution dans un notebook jupyter
import nest_asyncio
import asyncio

# Coroutines et tâches
Cette séance donne un aperçu des API de haut-niveau du module asyncio pour utiliser les coroutines et les tâches nouvelle notions à partir de Python 3.7.

## Coroutines
Les coroutines déclarées avec la syntaxe async/await sont la manière privilégiée d’écrire des applications asyncio. Par exemple, l’extrait de code suivant (requiert Python 3.7+) affiche « hello », attend 1 seconde et affiche ensuite « world » :

In [9]:
import asyncio

async def main():
     print('hello')
     await asyncio.sleep(1)
     print('world')


Faire `main()` appler la coroutine ne la planifie pas pour exécution :

In [10]:
main()

<coroutine object main at 0x000001A46200E0C0>

In [11]:
# par contre faut la lancer (un peu comme le start des Thread)
asyncio.run(main())

hello
world


Pour réellement exécuter une coroutine, asyncio fournit trois mécanismes principaux :

1. La fonction `asyncio.run()` pour exécuter la fonction « main() », le point d'entrée de haut-niveau (voir l'exemple ci-dessus).

2. Attendre une coroutine `await`. Le morceau de code suivant attend une seconde, affiche « hello », attend 2 secondes supplémentaires, puis affiche enfin « world » :

In [12]:
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")

    await say_after(1, 'hello') # ici await joue le role de l'activation de la coroutine say_after
    await say_after(2, 'world')

    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())

started at 20:30:47
hello
world
finished at 20:30:50


3. La fonction `asyncio.create_task()` pour exécuter de manière concurrente des coroutines en tant que tâches asyncio.

In [13]:
async def main():
    task1 = asyncio.create_task(
        say_after(1, 'hello'))

    task2 = asyncio.create_task(
        say_after(2, 'world'))

    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    await task1
    await task2

    print(f"finished at {time.strftime('%X')}")

In [14]:
asyncio.run(main())

started at 20:33:45
hello
world
finished at 20:33:47


*Comparer les temps des 2 dernier lancement de main*

# Les awaitable (attendables)

Un objet est dit attendable (awaitable en anglais, c.-à-d. qui peut être attendu) s'il peut être utilisé dans une expression await. Beaucoup d'API d'asyncio sont conçues pour accepter des attendables.

Il existe trois types principaux d'attendables : 
* les coroutines, 
* les tâches (remplacement de Future a partir de la version 3.7)
* et les futurs.

```python
#In Python 3.7+
task = asyncio.create_task(coro())
...
# This works in all Python versions but is less readable
task = asyncio.ensure_future(coro())
...
```


### Tâche

In [20]:

import asyncio

# coroutine async def
async def nested():
    return 42

def done(args):
    print("Callback", args)

# creation d'une tache puis await ou run
async def main():
    # Schedule nested() to run soon concurrently
    # with "main()".
    task = asyncio.create_task(nested())
    
    task.add_done_callback(done)

    # "task" can now be used to cancel "nested()", or
    # can simply be awaited to wait until it is complete:
    await task

    print(task.result())

asyncio.run(main())

Callback <Task finished name='Task-18' coro=<nested() done, defined at <ipython-input-20-78548a44dcd0>:4> result=42>
42
